# Load Twitter Dataset
Code is taken from prepareDataset.py
Goes through the DynamoDB tables created and creates a dataset of 2418 tweets.
The dataset is balanced: 1209 sarcastic, 1209 not sarcastic
The features are all created using Java program SentimentAnalyzer.java

This dataset will be used to train and test a model to detect sarcasm in tweets

In [1]:
from prepareDataset import *

In [2]:
tweets_search = getData("tweets_search")
tweets_stream = getData("tweets_sarcastic")
tweets = mergeDicts(tweets_search, tweets_stream)
print(len(tweets))

1209


In [3]:
sample = getRandomSample("tweets_hashtag", 1220)
print(len(sample))

1220


In [6]:
newSample = {}
randomSample = random.sample(range(1, 1220), 1209)
counter = 0
for k, v in sample.items():
    counter = counter + 1
    if counter not in randomSample:
        continue
    newSample[k] = v
sample = newSample
print(len(sample))

1209


In [7]:
tweets = mergeDicts(tweets, sample)
print(len(tweets))

2418


In [9]:
gateway = JavaGateway()
sa = gateway.entry_point.getSentimentAnalyzer()

In [10]:
ep = EmojiParse()

In [11]:
dynamodb = boto3.client('dynamodb')

In [15]:
for k, v in tweets.items():
    text, emoji, sentiment = ep.emoji_parse(k)

    sarcastic = v[0]
    id = v[1]

    emojiSentiment = emoji_sentiment_average(sentiment)
    positivity, adjacentContrast, maxPhrase, minPhrase, phraseContrast = sa.getFeatures(text)
    emojiSentenceContrast = emojiSentiment - positivity

    features = []
    features.append({"N" : str(positivity)})
    features.append({"N" : str(adjacentContrast)})
    features.append({"N" : str(maxPhrase)})
    features.append({"N" : str(minPhrase)})
    features.append({"N" : str(phraseContrast)})
    features.append({"N" : str(emojiSentiment)})
    features.append({"N" : str(emojiSentenceContrast)})

    response = dynamodb.put_item(
        TableName="twitter_dataset", 
        Item={
        "id" : {"N" : str(id)}, 
        "sarcastic" : {"BOOL" : sarcastic},
        "text" : {"S": text}, 
        "features": {"L": features},
        }
        )